In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Enhanced LSTM Model for Music Generation
Building upon the foundations of recurrent neural networks, this model leverages the power of bidirectional LSTMs and regularization techniques to create a more robust and capable system for music generation tasks.

## Enhanced Sampling for Generation
Implementing Temperature Sampling will allow control of the randomness of predictions. A higher temperature results in more random outputs, and a lower temperature makes the model's outputs more deterministic.


### Data Preparation

In [ ]:
import os
import numpy as np
from music21 import converter, instrument, note, chord

def read_midi(file):
    print("Loading Music File:", file)
    notes = []

    midi = converter.parse(file)
    parts = instrument.partitionByInstrument(midi)
    relevant_parts = parts.parts if parts else [midi]

    for part in relevant_parts:
        if 'Violin' in str(part.getInstrument()) or 'Violin' in str(part.partName):
            for element in part.recurse():
                if isinstance(element, note.Note):
                    notes.append((str(element.pitch), element.duration.quarterLength, element.offset))
                elif isinstance(element, chord.Chord):
                    notes.append(('.'.join(str(n) for n in element.normalOrder), element.duration.quarterLength, element.offset))
                elif isinstance(element, note.Rest):
                    notes.append(('rest', element.duration.quarterLength, element.offset))

    return notes


path = '/content/gdrive/MyDrive/Violin_Comp_Data/all_midi_files/'
files = [i for i in os.listdir(path) if i.endswith(".mid")]
notes_array = [read_midi(os.path.join(path, file)) for file in files]


### Encoding each unique note to an integer.

In [3]:
# Flatten
all_notes = [note for sequence in notes_array for note in sequence]

In [4]:
from fractions import Fraction

all_notes = [(pitch, float(duration) if isinstance(duration, Fraction) else duration,
              float(offset) if isinstance(offset, Fraction) else offset)
             for pitch, duration, offset in all_notes]


In [5]:
notes_array = [[(pitch, float(duration) if isinstance(duration, Fraction) else duration,
                 float(offset) if isinstance(offset, Fraction) else offset)
                for pitch, duration, offset in sequence]
               for sequence in notes_array]


In [6]:
note_to_int = {note: i for i, note in enumerate(sorted(set(all_notes)))}


In [7]:
input_sequences = []
output_notes = []
no_of_timesteps = 100

for notes in notes_array:
    for i in range(len(notes) - no_of_timesteps):
        input_seq = notes[i:i + no_of_timesteps]
        output_note = notes[i + no_of_timesteps]
        input_sequences.append([note_to_int[note] for note in input_seq])
        output_notes.append(note_to_int[output_note])

x_seq = np.array(input_sequences)
y_seq = np.array(output_notes)


### Initiating Train-Test Split & Reshaping Input for LSTM Model

In [8]:
from sklearn.model_selection import train_test_split

x_tr, x_val, y_tr, y_val = train_test_split(x_seq, y_seq, test_size=0.2, random_state=13)
x_tr = np.reshape(x_tr, (x_tr.shape[0], no_of_timesteps, 1))
x_val = np.reshape(x_val, (x_val.shape[0], no_of_timesteps, 1))


### Adjusting LSTM Model Complexity

In [9]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l2
from keras.optimizers import Adam

# Model Configuration
no_of_timesteps = 100
num_notes = len(note_to_int)

model = Sequential()

# First Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.001)), input_shape=(no_of_timesteps, 1)))

# Second LSTM Layer
model.add(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.001)))

# Third Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(64, kernel_regularizer=l2(0.001))))

# Dense Layer with Regularization
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(num_notes, activation='softmax'))

# Optimizer Configuration
optimizer = Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 100, 512)          528384    
 al)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 100, 128)          328192    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4915)              6

### Model Checkpoint

In [10]:
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model_lstm2.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

### Train Model

In [11]:
history = model.fit(x_tr, y_tr, epochs=50, batch_size=64, validation_data=(x_val, y_val), callbacks=[mc])

Epoch 1/10
3/3 [==============================] - ETA: 0s - loss: 9.4701 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 9.43121, saving model to best_model_violin.h5
3/3 [==============================] - 19s 2s/step - loss: 9.4701 - accuracy: 0.0000e+00 - val_loss: 9.4312 - val_accuracy: 0.0000e+00
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - ETA: 0s - loss: 9.3349 - accuracy: 0.0000e+00
Epoch 2: val_loss improved from 9.43121 to 9.38160, saving model to best_model_violin.h5
3/3 [==============================] - 5s 2s/step - loss: 9.3349 - accuracy: 0.0000e+00 - val_loss: 9.3816 - val_accuracy: 0.0000e+00
Epoch 3/10
3/3 [==============================] - ETA: 0s - loss: 9.1320 - accuracy: 0.0224
Epoch 3: val_loss improved from 9.38160 to 9.34674, saving model to best_model_violin.h5
3/3 [==============================] - 5s 2s/step - loss: 9.1320 - accuracy: 0.0224 - val_loss: 9.3467 - val_accuracy: 0.0000e+00
Epoch 4/10
3/3 [==============================] - ETA: 0s - loss: 8.8084 - accuracy: 0.0075
Epoch 4: val_loss improved from 9.34674 to 9.33394, saving model to best_model_violin.h5
3/3 [==============================] - 4s 1s/step - loss: 8.8084 - accuracy: 0.0075 - val_loss: 9.3339 - val_accuracy: 0.0000e+00
Epoch 5/10
3/3 [==============================] - ETA: 0s - loss: 8.33

### Load Best Model

In [12]:
from keras.models import load_model
model = load_model('best_model_lstm2.h5')

### Temperature Sampling

In [13]:
def sample_with_temperature(probabilities, temperature=1.0):
    if temperature <= 0:
        return np.argmax(probabilities)
    else:
        probabilities = np.asarray(probabilities).astype('float64')
        probabilities = np.log(probabilities + 1e-7) / temperature
        exp_probs = np.exp(probabilities)
        probabilities = exp_probs / np.sum(exp_probs)
        return np.random.choice(range(len(probabilities)), p=probabilities)


In [14]:
def generate_music(model, start_sequence, length=50, temperature=.96, lookback_length=100):
    prediction_output = []

    # Ensuring start_sequence is of length lookback_length
    if len(start_sequence) > lookback_length:
        start_sequence = start_sequence[-lookback_length:]
    elif len(start_sequence) < lookback_length:
        # Pad the sequence if it's too short
        start_sequence = [('rest', 0, 0)] * (lookback_length - len(start_sequence)) + start_sequence

    start_sequence_formatted = np.array([note_to_int[note] for note in start_sequence])

    for note_index in range(length):
        prediction_input = np.reshape(start_sequence_formatted, (1, lookback_length, 1))
        prob = model.predict(prediction_input)[0]
        index = sample_with_temperature(prob, temperature)
        predicted_note = x_int_to_note[index]
        prediction_output.append(predicted_note)

        # Update start_sequence_formatted for the next prediction
        start_sequence_formatted = np.append(start_sequence_formatted, [index])[-lookback_length:]

    return prediction_output


# Create the inverse mapping from integers back to note tuples
x_int_to_note = dict((number, note) for note, number in note_to_int.items())


The convert_to_midi function assumes that each note_info in prediction_output is a tuple with the structure (note, duration, offset), where:

- **note** can be either a note name (like 'C#4') or 'rest'.
- **duration** is the note's duration in quarterLength.
- **offset** is the note's offset.

In [15]:
from music21 import pitch

def midi_number_to_note_name(midi_number):
    return pitch.Pitch(midi=midi_number).nameWithOctave


In [16]:
from music21 import stream, instrument, note, chord

def convert_to_midi(prediction_output):
    midi_stream = stream.Stream()
    midi_stream.append(instrument.Violin())

    offset = 0
    for i, note_info in enumerate(prediction_output):
        try:
            note_name = note_info[0]
            # Check if note_name is a MIDI number and convert it
            if note_name.isdigit():
                note_name = midi_number_to_note_name(int(note_name))

            # Create note or rest
            if note_name != 'rest':
                new_note = note.Note(note_name)
            else:
                new_note = note.Rest()

            new_note.duration.quarterLength = note_info[1]
            new_note.offset = offset
            new_note.storedInstrument = instrument.Violin()
            midi_stream.append(new_note)
            offset += new_note.duration.quarterLength

        except Exception as e:
            print(f"Error processing note at position {i}: {note_info}. Error: {e}")

    midi_stream.write('midi', fp='lstm_music2.mid')

# Randomly select a starting sequence from x_val
random_index = np.random.randint(0, len(x_val))
start_sequence = x_val[random_index]

# Since start_sequence is currently encoded as integers, decode it back to note information
start_sequence_decoded = [x_int_to_note[note] for note in start_sequence.flatten()]

# Generate music based on the starting sequence
prediction_output = generate_music(model, start_sequence_decoded)
convert_to_midi(prediction_output)

1/1 [==============================] - 0s 77ms/step
